In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys

# Import API key
sys.path.append("..")
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mar Del Plata,AR,-38.0023,-57.5575,52.09,65,40,7.00,scattered clouds
1,1,Busselton,AU,-33.6500,115.3333,53.65,85,0,5.21,clear sky
2,2,Kibaya,TZ,-5.3000,36.5667,60.24,93,100,4.90,overcast clouds
3,3,Jucas,BR,-6.5253,-39.5275,76.66,74,57,5.68,broken clouds
4,4,Rikitea,PF,-23.1203,-134.9692,74.93,76,24,5.82,light rain


In [3]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 72
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Jucas,BR,-6.5253,-39.5275,76.66,74,57,5.68,broken clouds
4,4,Rikitea,PF,-23.1203,-134.9692,74.93,76,24,5.82,light rain
6,6,Timbiqui,CO,2.7717,-77.6654,78.49,86,96,5.88,light rain
8,8,Waingapu,ID,-9.6567,120.2641,77.31,88,100,7.23,moderate rain
13,13,Kuliyapitiya,LK,7.4688,80.0401,75.63,94,93,5.37,overcast clouds
15,15,Bobon,PH,12.5246,124.5636,76.78,92,96,5.48,overcast clouds
22,22,Jamestown,US,42.0970,-79.2353,76.01,51,0,11.50,clear sky
29,29,Albany,US,42.6001,-73.9662,76.46,46,99,5.35,overcast clouds
31,31,Guerrero Negro,MX,27.9769,-114.0611,76.03,45,0,12.77,clear sky
32,32,Bansbaria,IN,22.9719,88.3994,78.69,67,38,7.09,scattered clouds


In [5]:
#4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                141
City                   141
Country                141
Lat                    141
Lng                    141
Max Temp               141
Humidity               141
Cloudiness             141
Wind Speed             141
Current Description    141
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df=preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                141
City                   141
Country                141
Lat                    141
Lng                    141
Max Temp               141
Humidity               141
Cloudiness             141
Wind Speed             141
Current Description    141
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Jucas,BR,76.66,broken clouds,-6.5253,-39.5275,
4,Rikitea,PF,74.93,light rain,-23.1203,-134.9692,
6,Timbiqui,CO,78.49,light rain,2.7717,-77.6654,
8,Waingapu,ID,77.31,moderate rain,-9.6567,120.2641,
13,Kuliyapitiya,LK,75.63,overcast clouds,7.4688,80.0401,
15,Bobon,PH,76.78,overcast clouds,12.5246,124.5636,
22,Jamestown,US,76.01,clear sky,42.0970,-79.2353,
29,Albany,US,76.46,overcast clouds,42.6001,-73.9662,
31,Guerrero Negro,MX,76.03,clear sky,27.9769,-114.0611,
32,Bansbaria,IN,78.69,scattered clouds,22.9719,88.3994,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # print(hotels)
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except Exception as e:
        print(f"Exception of type {e} has occurred: lodging data not returned. Skipping...")
        pass    

Exception of type list index out of range has occurred: lodging data not returned. Skipping...
Exception of type list index out of range has occurred: lodging data not returned. Skipping...
Exception of type list index out of range has occurred: lodging data not returned. Skipping...
Exception of type list index out of range has occurred: lodging data not returned. Skipping...
Exception of type list index out of range has occurred: lodging data not returned. Skipping...
Exception of type list index out of range has occurred: lodging data not returned. Skipping...
Exception of type list index out of range has occurred: lodging data not returned. Skipping...
Exception of type list index out of range has occurred: lodging data not returned. Skipping...


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.count()

City                   141
Country                141
Max Temp               141
Current Description    141
Lat                    141
Lng                    141
Hotel Name             141
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))